# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [13]:
ed = Website("https://edwarddonner.com/")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/l

In [18]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [19]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/microsoft/phi-4',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-V3',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/reach-vb/2024-ai-timeline',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/fffiloni/LatentSync',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/trans

In [20]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [21]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details("https://huggingface.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
microsoft/phi-4
Updated
5 days ago
•
49.9k
•
1.12k
hexgrad/Kokoro-82M
Updated
7 days ago
•
11.4k
•
883
deepseek-ai/DeepSeek-V3
Updated
15 days ago
•
115k
•
1.75k
NovaSky-AI/Sky-T1-32B-Preview
Updated
about 8 hours ago
•
652
•
214
black-forest-labs/FLUX.1-dev
Updated
Aug 

In [42]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in portuguese and markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmicrosoft/phi-4\nUpdated\n5 days ago\n•\n49.9k\n•\n1.12k\nhexgrad/Kokoro-82M\nUpdated\n7 days ago\n•\n11.4k\n•\n883\ndeepseek-ai/DeepSeek-V3\nUpdated\n15 days ago\n•\n115k\n•\n1.75k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated\nabout 8 hours ago\n•\n652\n•\n214\nblack-forest-labs/FLUX.1-dev\nUpdate

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://huggingface.com/join'}]}


# 🤗 Welcome to Hugging Face! 🤗

## The AI Community Building the Future

At Hugging Face, we believe that artificial intelligence should be… well, friendly! We’re not just here to crunch numbers; we’re a cozy community where collaboration and creativity flow like data through a neural network. So, whether you're a curious coder, a data-driven dreamer, or a corporate titan, join us to transform the future of AI!

---

### 🤔 What Do We Do? 

- **Models Galore**: Ever wanted to browse over **400,000 models**? Us too! Model browsing is our cardio.
- **Datasets**: We have over **100,000 datasets** just lying around—like a digital treasure trove waiting for you to dive in!
- **Spaces**: Think of them as interactive playgrounds for creativity! (No swings, though—sorry!) 
- **Support for Everyone**: Whether you're with a corporate giant like Google or a sprightly little startup, we’ve got tailored solutions just for you!
  
### 🏢 Our Customers

More than **50,000 organizations** are on board with Hugging Face! Join the ranks of AI influencers:
- **Meta**: 2.06k models published and counting!
- **Amazon Web Services**: They're really having a “model” good time with us.
- **Google**: With 913 models, they keep us on our toes!

### 🎉 Company Culture

At Hugging Face, we don't just work; we *collaborate*! You'll find us sharing memes in the break room, not just models. Flexibility is our middle name. Well… technically, it's "Hugging," but you get the idea! 

- **Open Source Royalty**: We believe in sharing tools and tricks with the entire machine learning kingdom! 
- **Team Spirit**: Expect endless support, brainstorming sessions, and maybe the occasional snack competition. 

### 🚀 Careers at Hugging Face

Are you ready to embrace the future? Here’s what we’re looking for:
- **Curious Coders**: Love tinkering with models and datasets? We have a spot for you!
- **Language Lovers**: Got a knack for turning tech talk into human-friendly babble? Come aboard!
- **Chatty Creatives**: If you can mix art with data like peanut butter and jelly, we want to chat!

**Perks of Joining**:
- Competitive salaries that say, “You’re awesome!”
- An environment where your ideas have room to breathe (and grow!)
- An endless supply of virtual hugs! 🤗

---

### 🥳 Join Us!

Whether you’re here to create, collaborate, or contemplate (wow, we rhymed!), Hugging Face has something for everyone. Smash that **Sign Up** button and dive into the future of friendly AI! 

Check us out at [huggingface.co](https://huggingface.co) for more giggles, models, and datasets. 

### Remember: At Hugging Face, we're not just **friendly**; we're **AI-mazing**! 🎉

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [35]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [43]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}]}


# **Brochura do Hugging Face: Onde o Futuro da IA Funciona com Abraços!**

### **Quem Somos?**
Bem-vindo ao **Hugging Face**! Onde a inteligência artificial e o afeto se encontram! Sim, você não leu errado. Aqui, construímos o futuro um abraço por vez. A nossa comunidade de AI é tão unida que ao invés de “computação em nuvem”, deveríamos chamar de "nuvem de abraços"!

### **O Que Fazemos?**
Em nossa plataforma, você pode:

- **Modelos**: Temos modelos mais frescos que pão quente! Com mais de 400.000 opções à disposição, você vai encontrar o que precisa mais rápido do que consegue dizer "transformador" (ou deveria ser "tranformassor"?).
- **Datasets**: Nossa biblioteca contém dados mais pesados que um burrito gigante! Colete, compartilhe e brinque com mais de 100.000 datasets.
- **Applications (Espaços)**: Para inovação, somos como um parque de diversões. Imagine brinquedos que constroem aplicações para você! E não se preocupe, não tem fila para brincar.

### **Nossa Comunidade**
Mais de 50.000 organizações estão se abraçando neste espaço! Dentre elas, gigantes como **Amazon**, **Google** e **Microsoft**. E não se esqueça das maravilhas da nossa comunidade: nos ajudamos mutuamente como um bando de gansos voando em V, mas com mais IA e menos bico!

### **Diversão no Trabalho Com Nossa Cultura**
Se você achar que nossa comunidade se leva a sério, saiba que aqui também temos piadas internas, café que “faz mágica” e uma política de trabalho que é tão flexível quanto nosso abraço!

- **Inovação é a nossa meta**, mas temos espaço para aquela ideia bem maluca que vai fazer todos rirem.
- **Colaboração**: Sinta-se à vontade para dar seu palpite – mesmo se ele parecer um pouco “fora da caixa”. Na verdade, a caixa é só um espaço de armazenamento aqui!
- **Carreiras**: Se você quer ser o próximo grande abraço (ou ganhar um para começar), temos vagas abertas! Venha trabalhar onde cada linha de código é um passo em direção ao carinho (e à inteligência)!

### **Entre em Contato**
Se você estiver curioso ou estiver se perguntando como nosso modelo preferido de IA pode abraçar OS DESAFIOS DO MUNDO, não hesite! Entre em contato – estamos sempre prontos para receber novos abraços (ou perguntas) com um sorriso!

### **Viaje Com a Gente!**
Junte-se ao Hugging Face e participe de uma jornada de aprendizado em IA, onde o futuro é construído com criatividade, colaboração e muitos, muitos abraços!

---

**Abrace a inovação com Hugging Face! Onde a AI não é só INTELIGENTE, mas também AMIGA!** 😄🤗

In [37]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'company page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: Transforming AI Together

## Who We Are
Hugging Face is at the forefront of the AI revolution, dedicated to creating a collaborative platform for the global machine learning community. Our mission is to empower users to create, discover, and collaborate on models and datasets, driving innovation in artificial intelligence.

## Our Products
- **Models**: Access over 400,000 high-quality models for a myriad of applications.
- **Datasets**: Browse through 100,000 datasets to kickstart your projects.
- **Spaces**: Host and share engaging ML applications effortlessly.
- **Enterprise Solutions**: Tailored solutions with dedicated support and enterprise-grade security for businesses looking to scale their AI efforts.

## Join the Community
More than 50,000 organizations, including major tech giants like Google, Microsoft, and Amazon Web Services, rely on Hugging Face. Collaborate with us to build the future of machine learning.

## Company Culture
At Hugging Face, we thrive on collaboration and community spirit. Our open-source roots foster an inclusive environment where innovation flourishes. We celebrate diverse ideas and perspectives, paving the way for groundbreaking advancements in AI.

## Careers at Hugging Face
We are always on the lookout for talent that aligns with our vision. A career at Hugging Face means being part of a pioneering team that values creativity, integrity, and collaboration. Current openings include software engineers, machine learning researchers, and product managers. Explore your potential with us!

### Highlights of Working with Us:
- **Work on cutting-edge technology**: Engage with the latest in AI and machine learning.
- **Flexible working environment**: We embrace remote work and promote a healthy work-life balance.
- **Community-driven**: Be part of a larger mission to democratize AI.

## Get In Touch
To learn more about Hugging Face, our community, or explore career opportunities, visit our website: [Hugging Face](https://huggingface.co)

Join us in building the future of AI, one model at a time!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>